# Dependencies

In [11]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pandas as pd
import random
import time

# Retrieve the dataframes from sources

In [7]:
import pandas as pd
import random


df_dim_static = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'Dim - Static')
df_sortation_material_type = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'Sortation Material Type')
df_units_installed = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'units installed')
df_orders = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'Replynish - Orders')
df_sortation_data = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'fact - Sortation Data')
df_financial_cost = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'Financial - Cost')
df_financial_sales = pd.read_excel("Cru3 Data.xlsx", sheet_name = 'Financial - Sales')

# Fucntions

## Geo-location boundaries

In [8]:
VIENNA_LAT_MIN, VIENNA_LAT_MAX = 46.36, 49.02
VIENNA_LON_MIN, VIENNA_LON_MAX = 9.53, 17.16

## Fucntions

In [9]:
geolocator = Nominatim(user_agent="vienna_fix", timeout=10)


def swap_characters(entity):
    swapped = ''
    for char in entity:
        if char.isdigit():
            swapped += chr(ord('A') + int(char))  # 0 -> A, 1 -> B, ..., 9 -> J
        elif char.isalpha():
            swapped += str(ord(char.upper()) - ord('A'))  # A -> 0, B -> 1, ..., Z -> 25
        else:
            swapped += char
    return swapped

def is_within_vienna(lat, lon):
    return VIENNA_LAT_MIN <= lat <= VIENNA_LAT_MAX and VIENNA_LON_MIN <= lon <= VIENNA_LON_MAX

def generate_random_vienna_coord():
    return (
        random.uniform(VIENNA_LAT_MIN, VIENNA_LAT_MAX),
        random.uniform(VIENNA_LON_MIN, VIENNA_LON_MAX)
    )

def reverse_geocode_vienna(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en')
        if location and location.raw:
            address = location.raw.get('address', {})
            return {
                'state': address.get('state', ''),
                'location address': location.address,
                'city': address.get('city', '') or address.get('town', '') or address.get('village', ''),
                'Region': address.get('region', ''),
                'Region Name': address.get('county', '') or address.get('state_district', '')
            }
    except Exception:
        return {}
    return {}
def relocate_to_vienna(row):
    if not is_within_vienna(row['lat'], row['lon']):
        new_lat, new_lon = generate_random_vienna_coord()
        row['lat'] = new_lat
        row['lon'] = new_lon

        metadata = reverse_geocode_vienna(new_lat, new_lon)
        for key, value in metadata.items():
            row[key] = value
        
        time.sleep(1)  # prevent API throttling
    return row


# Tables

## Dim Static

In [ ]:
# Entity randomization
sampled_entities = df_dim_static['Entity'].sample(20, random_state=42).tolist()
df_dim_static = df_dim_static[df_dim_static['Entity'].isin(sampled_entities)]
df_dim_static["Entity"] = df_dim_static["Entity"].apply(swap_characters)
df_dim_static["account name"] = df_dim_static["account name"].apply(swap_characters)

# Geozone relocation
df_dim_static = df_dim_static.apply(relocate_to_vienna, axis=1)
df_dim_static['Region'] = df_dim_static['Region Name']
df_dim_static['fulfillment party'] = 'Company X'
df_dim_static['Alias'] = df_dim_static['Entity']

# Numerical data
df_dim_static['Weeks in Service'] = df_dim_static['Weeks in Service'].apply(lambda x: round(x * random.uniform(0.7, 0.9), 0) if pd.notnull(x) else x)
df_dim_static['Pick-up Frequency'] = df_dim_static['Pick-up Frequency'].apply(lambda x: round(x * random.uniform(0.7, 0.9), 2) if pd.notnull(x) else x)

# Unavailable data
df_dim_static = df_dim_static.replace("", "Not Available")
df_dim_static = df_dim_static.fillna("Not Available")

df_dim_static.reset_index(drop=True, inplace=True)
df_dim_static

,Entity,lat,lon,state,location address,fulfillment party,account name,Alias,Region,Weeks in Service,Pick-up Frequency,Population,city,Region Name
0,0013,48.261501,10.305468,Bavaria,"St 2019, Glaserhof, Kiesberg, Deisenhausen, Kr...",Company X,CCBC,0013,Landkreis Günzburg,16.0,0.20,25464,Deisenhausen,Landkreis Günzburg
1,0073,47.129519,12.752954,Salzburg,"Hohe Dock Steig, Fusch an der Großglocknerstra...",Company X,CCBC,0073,Bezirk Zell am See,49.0,0.42,31578,Fusch an der Großglocknerstraße,Bezirk Zell am See
2,0100,47.203268,10.882400,Tyrol,"Hauptstraße, Oetzerau, Oetz, Bezirk Imst, Tyro...",Company X,CCBC,0100,Bezirk Imst,67.0,0.59,38006,Oetzerau,Bezirk Imst
3,0247,47.458145,9.681256,Vorarlberg,"Königswiesen, Lustenau, Bezirk Dornbirn, Vorar...",Company X,CCBC,0247,Bezirk Dornbirn,56.0,0.15,9148,Lustenau,Bezirk Dornbirn
4,0331,48.576235,16.750478,Lower Austria,"Am Steinberg, Katastralgemeinde Gösting, Ziste...",Company X,CCBC,0331,Bezirk Gänserndorf,22.0,0.59,30299,Zistersdorf,Bezirk Gänserndorf
5,0374,46.950896,15.974936,Styria,"Johnsdorferstraße, Johnsdorf, Fehring, Bezirk ...",Company X,CCBC,0374,Bezirk Südoststeiermark,38.0,0.29,15955,Fehring,Bezirk Südoststeiermark
6,0489,48.058176,10.674465,Bavaria,"13, Pirmin-Klaunzler-Straße, Amberg, Verwaltun...",Company X,CCBC,0489,Landkreis Unterallgäu,28.0,1.18,20748,Amberg,Landkreis Unterallgäu
7,0839,47.711069,12.476257,Bavaria,"B 305, Brem, Oberwössen, Unterwössen, Landkrei...",Company X,CCBC,0839,Landkreis Traunstein,21.0,0.29,31423,Oberwössen,Landkreis Traunstein
8,1342,47.129639,16.324468,Burgenland,"Punitzer Straße, Pagelberg, Punitz, Tobaj, Bez...",Company X,CCBC,1342,Bezirk Güssing,32.0,1.23,50210,Punitz,Bezirk Güssing
9,4808,47.852778,15.451209,Lower Austria,"Sandbühel, Katastralgemeinde Weidenaurotte, Tü...",Company X,BIC,4808,Bezirk Lilienfeld,60.0,0.24,44822,Türnitz,Bezirk Lilienfeld


## Units

In [6]:
df_units_installed = df_units_installed[df_units_installed['Entity'].isin(sampled_entities)]
df_units_installed["Entity"] = df_units_installed["Entity"].apply(swap_characters)
df_units_installed.reset_index(drop=True, inplace=True)
df_units_installed



,Entity,Time,installed units
0,EIAI,2023-05-16,1
1,EIAI,2023-05-17,1
2,EIAI,2023-05-18,1
3,EIAI,2023-05-19,1
4,EIAI,2023-05-20,1
...,...,...,...
10657,ICFJ,2025-07-26,1
10658,ICFJ,2025-07-27,1
10659,ICFJ,2025-07-28,1
10660,ICFJ,2025-07-29,1


## Orders

In [7]:
df_orders["Entity"] = df_orders["Entity"].astype(str)
df_orders = df_orders[df_orders['Entity'].isin(sampled_entities)]
df_orders["Entity"] = df_orders["Entity"].apply(swap_characters)

#numerical data
df_orders['Value'] = df_orders['Value'].apply(lambda x: round(x * random.uniform(0.7, 1.3), 0) if pd.notnull(x) else x)
df_orders.reset_index(drop=True, inplace=True)
df_orders


,Entity,Date,fulfilled on,Attribute.1,Attribute.2,Attribute.3,Value
0,EIAI,2023-05-22,NaN,collected,CRU3 - Orders,NaN,1.0
1,EIAI,2023-05-30,NaN,collected,CRU3 - Orders,NaN,1.0
2,EIAI,2023-06-07,NaN,collected,CRU3 - Orders,NaN,1.0
3,EIAI,2023-06-15,NaN,collected,CRU3 - Orders,NaN,1.0
4,EIAI,2023-06-28,NaN,collected,CRU3 - Orders,NaN,1.0
...,...,...,...,...,...,...,...
2905,ICCG,2025-04-28,NaN,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging,2.0
2906,ICCG,2025-04-28,NaN,collected,collected HDPE #2 Containers - Mixed (Plastic),collected Home Cleaning/Beauty Bottles,2.0
2907,ICCG,2025-04-28,NaN,collected,collected Aluminum Cans (Nonferrous Metal),collected Aluminum,2.0
2908,ICCG,2025-04-28,NaN,collected,collected PET #1 Thermoforms (Plastic),collected Clear Plastic Containers,2.0


## Sortation Data

In [8]:
df_sortation_data["Entity"] = df_sortation_data["Entity"].astype(str)
df_sortation_data = df_sortation_data[df_sortation_data['Entity'].isin(sampled_entities)]
df_sortation_data["Entity"] = df_sortation_data["Entity"].apply(swap_characters)
df_sortation_data.reset_index(drop=True, inplace=True)

# numerical data
df_sortation_data['Weight'] = df_sortation_data['Weight'].apply(lambda x: round(x * random.uniform(0.1, 1.3), 0) if pd.notnull(x) else x)
df_sortation_data

,Entity,Time,Weight,Weight Material Type.1,Weight Material Type.2,Weight Material Type.3
0,BDEC,2024-06-13,0.0,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging
1,BDEC,2024-06-27,0.0,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging
2,BDEC,2024-07-04,0.0,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging
3,BDEC,2024-07-18,0.0,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging
4,BDEC,2024-07-25,0.0,collected,collected Polypropylene #5 (Plastic),collected Rigid Plastic Food Packaging
...,...,...,...,...,...,...
5673,ICIC,2025-03-05,86.0,collected,collected LDPE #4 Film (Plastic),collected Plastic Bags & Film
5674,ICIC,2025-04-23,118.0,collected,collected LDPE #4 Film (Plastic),collected Plastic Bags & Film
5675,ICIC,2025-05-07,28.0,collected,collected LDPE #4 Film (Plastic),collected Plastic Bags & Film
5676,ICIC,2025-07-09,57.0,collected,collected LDPE #4 Film (Plastic),collected Plastic Bags & Film


## Financial cost

In [9]:
df_financial_cost['Entity'] = 'Company X'
df_financial_cost = df_financial_cost[df_financial_cost['Key Column'] % 2 == 0]

# Numeric data
df_financial_cost['Amount'] = df_financial_cost['Amount'].apply(lambda x: round(x * random.uniform(0.1, 1.3), 2) if pd.notnull(x) else x)

# Source Number and Name
df_financial_cost['Source Number'] = df_financial_cost['Source Number'].apply(lambda x: f"SN-{random.randint(1000, 9999)}")
df_financial_cost['Source Name'] = df_financial_cost['Source Name'].apply(lambda x: f"Source {random.randint(1, 100)}")
df_financial_cost.reset_index(drop=True, inplace=True)

# Account Number
df_financial_cost['Account Numer'] = df_financial_cost['Account Numer'].apply(lambda x: f"AN-{random.randint(1000, 9999)}")
df_financial_cost

,Entity,Time,Key Column,Document Type,Account Numer,Source Number,Source Name,Amount,Year,Date
0,Company X,2022-12-28,2420666,2,AN-5781,SN-5358,Source 86,295.29,2022,2022-12-28
1,Company X,2022-12-28,2420688,2,AN-3211,SN-2453,Source 98,157.79,2022,2022-12-28
2,Company X,2023-01-03,2496704,2,AN-5348,SN-2015,Source 69,86.17,2023,2023-01-03
3,Company X,2023-01-05,2557156,2,AN-3597,SN-3327,Source 2,26.18,2023,2023-01-05
4,Company X,2023-01-19,2638782,2,AN-2307,SN-9783,Source 4,23.32,2023,2023-01-19
...,...,...,...,...,...,...,...,...,...,...
276,Company X,2025-07-01,18847480,2,AN-6319,SN-2552,Source 1,22.98,2025,2025-07-01
277,Company X,2025-06-29,18860116,2,AN-2553,SN-9799,Source 25,1964.25,2025,2025-06-29
278,Company X,2025-07-02,18860118,2,AN-1720,SN-2507,Source 89,12210.77,2025,2025-07-02
279,Company X,2025-07-03,18880310,2,AN-2884,SN-3659,Source 65,5175.22,2025,2025-07-03


## Financial Sales

In [10]:
df_financial_sales['Entity'] = 'Company X'
df_financial_sales = df_financial_sales[df_financial_sales['Key Column'] % 2 == 0]

# Numeric data
df_financial_sales['Amount'] = df_financial_sales['Amount'].apply(lambda x: round(x * random.uniform(0.1, 1.3), 2) if pd.notnull(x) else x)

# Source Number and Name
df_financial_sales['Facility Code'] = df_financial_sales['Facility Code'].apply(lambda x: f"FC-{random.randint(1000, 9999)}")
df_financial_sales['Customer Name'] = df_financial_sales['Customer Name'].apply(lambda x: f"Source {random.randint(1, 100)}")
df_financial_sales.reset_index(drop=True, inplace=True)

# Account Number
df_financial_sales['Account Number'] = df_financial_sales['Account Number'].apply(lambda x: f"AN-{random.randint(1000, 9999)}")
df_financial_sales

,Entity,Time,Key Column,Document Type,Amount,Account Number,Facility Code,Customer Name,Year,Month,Date
0,Company X,2022-12-16,2353370,2,35191.45,AN-7176,FC-8325,Source 96,2022,December,2022-12-16
1,Company X,2022-12-16,2353432,2,28364.09,AN-7691,FC-2200,Source 99,2022,December,2022-12-16
2,Company X,2023-04-01,3338736,2,7398.59,AN-8224,FC-4661,Source 95,2023,April,2023-04-01
3,Company X,2023-04-27,3429854,2,270410.20,AN-3540,FC-5816,Source 3,2023,April,2023-04-27
4,Company X,2023-08-28,4686278,2,5904.01,AN-7670,FC-7229,Source 7,2023,August,2023-08-28
...,...,...,...,...,...,...,...,...,...,...,...
62,Company X,2024-12-31,17136944,0,-1167.57,AN-5951,FC-5539,Source 98,2024,December,2024-12-31
63,Company X,2025-05-14,18305342,2,2503.82,AN-4973,FC-8134,Source 91,2025,May,2025-05-14
64,Company X,2025-05-21,18306630,2,240.71,AN-9054,FC-6486,Source 20,2025,May,2025-05-21
65,Company X,2025-06-11,18327690,2,435.01,AN-6419,FC-2092,Source 65,2025,June,2025-06-11


# Saving to Excel

In [11]:
with pd.ExcelWriter("D6 Inc randomized data.xlsx", engine="xlsxwriter") as writer:
    df_dim_static.to_excel(writer, sheet_name="Dim - Static", index=False)
    df_sortation_material_type.to_excel(writer, sheet_name="Sortation Material Type", index=False)
    df_units_installed.to_excel(writer, sheet_name="units installed", index=False)
    df_orders.to_excel(writer, sheet_name="Replynish - Orders", index=False)
    df_sortation_data.to_excel(writer, sheet_name="fact - Sortation Data", index=False)
    df_financial_cost.to_excel(writer, sheet_name="Financial - Cost", index=False)
    df_financial_sales.to_excel(writer, sheet_name="Financial - Sales", index=False)